In [ ]:
# feature engineering
# Conny Lin | June 14, 2020

In [1]:
# updated June 14, 2020
# computer dependent local variable settings 
import socket, os
# check which computer this code is running on
hostname = socket.gethostname().split('.')[0]
# set local path settings based on computer host
if hostname == 'PFC':
    pylibrary = ['/Users/connylin/Dropbox/Code/proj/']
    pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone'
    datapath = os.path.join(pCapstone,'data','nutcracker_sample_1Meach.csv')
elif hostname == 'Angular Gyrus':
    pylibrary = ['/Users/connylin/Code/proj/']
    pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone'
    datapath = os.path.join(pCapstone,'data','nutcracker_sample_1Meach.csv')
else:
    assert False, 'host computer not regonized'
    
# import standard libraries
import sys, pickle, socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# load local libraries
for path in pylibrary:
    if path not in sys.path:
        sys.path.insert(1, path)
import brainstation_capstone
from brainstation_capstone.etl.datatransform import Nutcracker


In [2]:
# ML specific 
outputfolder = os.path.join(pCapstone,'ml_feature_eng')
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression

Many of the features in nutcracker are calculated to represent similar things

time measures:
* time
* persistence

size measures:
* area

width measures:
* midline
* morphwidth
* width
* relwidth

length measures:
* length
* rellength

angle measures:
* aspect
* relaspect
* kink
* curve
* orient

movement speed measures:
* speed
* crab
* angular

movement dir measures: 
* bias
* dir
* vel_x
* vel_y



In [3]:
from brainstation_capstone.db import chor_legend
# create reference table
chor_legend = chor_legend.get_chor_legend()
chor_legend

,call,name,type,category,description
0,t,time,measure,time,time of the frame
1,f,frame,measure,time,the frame number
2,p,persistence,measure,time,length of time object is tracked
3,D,id,measure,id,the object ID
4,n,number,measure,sample_size,the number of objects tracked
5,N,goodnumber,measure,sample_size,the number of objects passing the criteria gi...
6,e,area,measure,area,body area
7,m,midline,measure,width,length measured along the curve of object
8,M,morphwidth,measure,width,mean width of body about midline
9,w,width,measure,width,width of the rectangle framing the body


use OLS 

In [6]:
# list features
NC = Nutcracker(datapath)
# get machine learning input data
X_train, X_test, y_train, y_test = NC.mldata()
X_columns = np.array(NC.names['X']).transpose()

In [7]:
# OLS to view the best features
C=0.05
random_state = 318
max_iter=1000
OLS = LogisticRegression(C=C, random_state=random_state, max_iter=max_iter)
OLS.fit(X_train, y_train)
# get coefficient
X_logistic_coef = OLS.coef_

/Users/connylin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [34]:
# set dataframe
top_features = pd.DataFrame(X_columns, columns=['features'])
# put int OLS_coef
top_features['OLS_coef'] = X_logistic_coef[0]
# put in absolute
top_features['OLS_coef_abs'] = np.absolute(top_features['OLS_coef'].values)
# join table with category
top_features = top_features.merge(chor_legend[['name','category']], 
                   left_on='features', right_on='name', how='left')
top_features.drop(columns='name',inplace=True)
top_features.sort_values(ascending=False, inplace=True, by=['OLS_coef_abs'])
top_features

,features,OLS_coef,OLS_coef_abs,category
5,width,-13.761015,13.761015,width
9,aspect,-10.941442,10.941442,angle
6,relwidth,3.617541,3.617541,width
10,relaspect,2.827076,2.827076,angle
8,rellength,-2.393815,2.393815,length
4,morphwidth,2.076456,2.076456,width
7,length,2.056058,2.056058,length
2,area,-0.712834,0.712834,area
13,speed,0.684202,0.684202,speed
20,crab,-0.585187,0.585187,speed


as expected, time doesn't look like a good predictive feature. drop them.

Use variance inflation factor to detect multicolinearity. Use trained split and scaled data.

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# get data frame 
df = pd.DataFrame(X_train, columns=X_columns)

VIF = pd.Series([variance_inflation_factor(df.values, i) 
           for i in range(df.shape[1])], index=df.columns)

In [28]:
VIF

time             5.816375
persistence      2.637530
area           139.898284
midline        457.248200
morphwidth      52.038697
width          114.543742
relwidth       263.506779
length         380.273174
rellength      152.491069
aspect          83.754391
relaspect      185.980740
kink             6.366266
curve           17.415263
speed            5.500643
angular          1.857357
bias             3.217497
dir              1.025195
vel_x            1.000438
vel_y            1.005443
orient           1.181026
crab             2.499766
dtype: float64

In [37]:
# join with table
vif_df = pd.DataFrame({'features':VIF.index.values, 'VIF': VIF.values})
# join table with category
top_features = top_features.merge(vif_df, how='left',on='features', right_index=False)
top_features

,features,OLS_coef,OLS_coef_abs,category,VIF
0,width,-13.761015,13.761015,width,114.543742
1,aspect,-10.941442,10.941442,angle,83.754391
2,relwidth,3.617541,3.617541,width,263.506779
3,relaspect,2.827076,2.827076,angle,185.980740
4,rellength,-2.393815,2.393815,length,152.491069
5,morphwidth,2.076456,2.076456,width,52.038697
6,length,2.056058,2.056058,length,380.273174
7,area,-0.712834,0.712834,area,139.898284
8,speed,0.684202,0.684202,speed,5.500643
9,crab,-0.585187,0.585187,speed,2.499766


Keep only top for each category, drop time

In [59]:
# for each category, get the top ols_coeff
categories = top_features['category'].value_counts().index.values
# remove time 
categories = categories[categories != 'time']

In [60]:
best_feature_by_category = []
for c in categories:
    i = top_features['category']== c
    d = top_features.loc[i, ['features','category','OLS_coef_abs']]
    j = d['OLS_coef_abs'].argmax()
    best_feature = d['features'].iloc[j]
    best_feature_by_category.append(best_feature)
best_feature_by_category
# remove time

['speed', 'aspect', 'width', 'rellength', 'dir', 'area']

Recorded ['speed', 'aspect', 'width', 'rellength', 'dir', 'area'] top category in Nutcracker object in datatransform.py 

In [61]:
# try VIF again with just top features
from statsmodels.stats.outliers_influence import variance_inflation_factor
# get data frame 
df = pd.DataFrame(X_train, columns=X_columns)
df = df[best_feature_by_category].copy()

VIF = pd.Series([variance_inflation_factor(df.values, i) 
           for i in range(df.shape[1])], index=df.columns)
VIF

speed         3.652589
aspect       22.244330
width        42.550239
rellength    13.018498
dir           1.013721
area         20.659303
dtype: float64

In [6]:
# try with OLS
# list features
NC = Nutcracker(datapath)
# get machine learning input data
X_train, X_test, y_train, y_test = NC.mldata()
X_columns = np.array(NC.names['X']).transpose()


In [75]:
# get index
i = []
for c in best_feature_by_category:
    i.append(np.where(X_columns == c)[0][0])
X_columns_reduced = X_columns[i]
X_train_reduced = X_train[:,i]
X_test_reduced = X_test[:,i]
X_train_reduced.shape

(1600000, 6)

In [76]:
# OLS to view the best features
C=0.05
random_state = 318
max_iter=1000
OLS = LogisticRegression(C=C, random_state=random_state, max_iter=max_iter)
OLS.fit(X_train_reduced, y_train)
# get coefficient
X_logistic_coef = OLS.coef_

In [80]:
y_predicted = OLS.predict(X_test_reduced)
print(f'train score: {OLS.score(X_train_reduced,y_train)}')
print(f'test score: {OLS.score(X_test_reduced,y_test)}')

train score: 0.715986875
test score: 0.7158025


Ok. that's wayy too much reduction. Previously the prediction is 0.86.

In [82]:
for c in X_columns:
    print(f"'{c}',",end=' ')


'time', 'persistence', 'area', 'midline', 'morphwidth', 'width', 'relwidth', 'length', 'rellength', 'aspect', 'relaspect', 'kink', 'curve', 'speed', 'angular', 'bias', 'dir', 'vel_x', 'vel_y', 'orient', 'crab', 

In [ ]:
# take just time out
# get index
feature_selection = ['area', 'midline', 'morphwidth', 'width', 'relwidth', 'length', 'rellength', 'aspect', 'relaspect', 'kink', 'curve', 'speed', 'angular', 'bias', 'dir', 'vel_x', 'vel_y', 'orient', 'crab']
i = []
for c in feature_selection:
    i.append(np.where(X_columns == c)[0][0])
Xr_column = X_columns[i]
Xr_train = X_train[:,i]
Xr_test = X_test[:,i]
Xr_train.shape

# OLS to view the best features
C=0.05
random_state = 318
max_iter=1000
OLS = LogisticRegression(C=C, random_state=random_state, max_iter=max_iter)
OLS.fit(Xr_train, y_train)
print(f'train score: {OLS.score(Xr_train,y_train)}')
print(f'test score: {OLS.score(Xr_test,y_test)}')
# get coefficient
X_logistic_coef = OLS.coef_

# set dataframe
top_features = pd.DataFrame(Xr_column, columns=['features'])
# put int OLS_coef
top_features['OLS_coef'] = X_logistic_coef[0]
# put in absolute
top_features['OLS_coef_abs'] = np.absolute(top_features['OLS_coef'].values)
# join table with category
top_features.sort_values(ascending=False, inplace=True, by=['OLS_coef_abs'])
top_features

In [ ]:
# try PCA
